In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from tqdm import tqdm
import torchvision 
from torchvision import datasets, transforms, models
from torchvision.models import resnet18, resnet50, resnet152, efficientnet_v2_m, convnext_base, wide_resnet101_2, vgg19_bn, regnet_x_32gf, swin_b, maxvit_t
import matplotlib.pyplot as plt
import openslide
import time
import os
import copy
import pandas as pd
from torch.utils.data import DataLoader, Dataset
import torch.nn.functional as F
from sklearn.model_selection import KFold

In [ ]:
ls /home/ngsci/datasets/brca-psj-path/contest-phase-2/

In [2]:
ls /home/ngsci/datasets/covid-psj-xray/v2/xrays/p000000/

e6c636da7360ec300c0e120316e10d287efe4de78891d43730b5c714b70f542d.dcm*


In [2]:
def to_device(model):
    if len(num_gpus) > 1:
        model = torch.nn.DataParallel(model, device_ids=num_gpus)
        model = model.module
    model = model.to(device)
    return model

In [3]:
from sklearn.model_selection import KFold

def train_model(model, model_name, dataloaders, criterion, optimizer, num_classes, num_epochs, is_inception, lr, df_train):
    since = time.time()
    n_splits = 5
    kf = KFold(n_splits=n_splits, shuffle=True)

    # Initialize variables to store results
    avg_training_loss = 0.0
    avg_validation_loss = 0.0
    avg_training_accuracy = 0.0
    avg_validation_accuracy = 0.0

    for fold, (train_idx, val_idx) in enumerate(kf.split(df_train)):
        print(f"Fold {fold+1}/{n_splits}")
        print('-' * 10)

        # Splitting the indices for training and validation
        train_biopsy_paths = [biopsy_feature_paths_train[i] for i in train_idx]
        train_labels = [labels_train[i] for i in train_idx]
        val_biopsy_paths = [biopsy_feature_paths_train[i] for i in val_idx]
        val_labels = [labels_train[i] for i in val_idx]

        # Creating new datasets for the current fold
        train_dataset_fold = BiopsyDataset(train_biopsy_paths, train_labels)
        val_dataset_fold = BiopsyDataset(val_biopsy_paths, val_labels)

        # Creating dataloaders for the current fold
        train_dataloader_fold = DataLoader(train_dataset_fold, batch_size=4, shuffle=True)
        val_dataloader_fold = DataLoader(val_dataset_fold, batch_size=4, shuffle=False)

        best_loss = float('inf')
        best_model_wts = copy.deepcopy(model.state_dict())

        for epoch in range(num_epochs):
            print("Epoch {}/{}".format(epoch + 1, num_epochs))

            for phase in ["train", "valid"]:
                if phase == "train":
                    model.train()
                else:
                    model.eval()

                running_loss = 0.0
                running_corrects = 0

                for inputs, labels in dataloaders[phase]:
                    inputs = inputs.to(device)
                    labels = labels.to(device)
                    optimizer.zero_grad()

                    with torch.set_grad_enabled(phase == "train"):
                        if is_inception and phase == "train":
                            outputs, aux_outputs = model(inputs)
                            loss1 = criterion(outputs, labels)
                            loss2 = criterion(aux_outputs, labels)
                            loss = loss1 + 0.4 * loss2
                        else:
                            outputs = model(inputs)
                            loss = criterion(outputs, labels)
                        _, preds = torch.max(outputs, 1)

                        if phase == "train":
                            loss.backward()
                            optimizer.step()

                    running_loss += loss.item() * inputs.size(0)
                    running_corrects += torch.sum(preds == labels.data)

                epoch_loss = running_loss / len(dataloaders[phase].dataset)
                epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)

                if phase == "valid" and epoch_loss < best_loss:
                    best_loss = epoch_loss
                    best_model_wts = copy.deepcopy(model.state_dict())

            avg_training_loss += epoch_loss if phase == 'train' else avg_training_loss
            avg_validation_loss += epoch_loss if phase == 'valid' else avg_validation_loss
            avg_training_accuracy += epoch_acc if phase == 'train' else avg_training_accuracy
            avg_validation_accuracy += epoch_acc if phase == 'valid' else avg_validation_accuracy

    # Loading best model weights
    model.load_state_dict(best_model_wts)

    # Calculate average loss and accuracy
    avg_training_loss /= n_splits
    avg_validation_loss /= n_splits
    avg_training_accuracy /= n_splits
    avg_validation_accuracy /= n_splits

    print(f'Average Training Loss: {avg_training_loss:.4f}, Average Validation Loss: {avg_validation_loss:.4f}')
    print(f'Average Training Accuracy: {avg_training_accuracy:.4f}, Average Validation Accuracy: {avg_validation_accuracy:.4f}')

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))

    return model


In [4]:
def initialize_model(model_name, num_classes, feature_extract, use_pretrained=True):
    model_ft = None
    input_size = 0
    if model_name == "resnet_18":
        model_ft = resnet18(weights=None)
        checkpoints = torch.load("pretrained_weights/resnet18-f37072fd.pth")
        model_ft.load_state_dict(checkpoints)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.fc.in_features
        model_ft.fc = nn.Linear(num_ftrs, num_classes)
        input_size = 224

    if model_name == "resnet50":
        model_ft = resnet50(weights=None)
        checkpoints = torch.load("pretrained_weights/resnet50-0676ba61.pth")
        model_ft.load_state_dict(checkpoints)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.fc.in_features
        model_ft.fc = nn.Linear(num_ftrs, num_classes)
        input_size = 224
    
    if model_name == "wide_resnet101":
        """Wide Resnet101 V2"""
        model_ft = wide_resnet101_2(weights=None)
        checkpoints = torch.load('pretrained_weights/wide_resnet101_2-d733dc28.pth')
        model_ft.load_state_dict(checkpoints)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.fc.in_features
        model_ft.fc = nn.Linear(num_ftrs, num_classes)
        input_size = 224

    if model_name == "resnet152":
        """Resnet152"""
        model_ft = resnet152(weights=None)
        checkpoints = torch.load('pretrained_weights/resnet152-394f9c45.pth')
        model_ft.load_state_dict(checkpoints)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.fc.in_features
        model_ft.fc = nn.Linear(num_ftrs, num_classes)
        input_size = 224

    if model_name == "efficientnet":
        """EFFICIENTNET_V2_M"""
        model_ft = efficientnet_v2_m(weights=None)
        checkpoints = torch.load('pretrained_weights/efficientnet_v2_m-dc08266a.pth')
        model_ft.load_state_dict(checkpoints)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.classifier[1].in_features
        model_ft.classifier[1] = nn.Linear(num_ftrs, num_classes)
        input_size = 224

    if model_name == "convnext":
        """convnext_base"""
        model_ft = convnext_base(weights=None)
        checkpoints = torch.load('pretrained_weights/convnext_base-6075fbad.pth')
        model_ft.load_state_dict(checkpoints)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.classifier[2].in_features
        model_ft.classifier[2] = nn.Linear(num_ftrs, num_classes)
        input_size = 224
    
    if model_name == "vgg":
        """VGG19_bn"""
        model_ft = vgg19_bn(weights=None)
        checkpoints = torch.load('pretrained_weights/vgg19_bn-c79401a0.pth')
        model_ft.load_state_dict(checkpoints)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.classifier[6].in_features
        model_ft.classifier[6] = nn.Linear(num_ftrs,num_classes)
        input_size = 224

    if model_name == "regnet":
        """regnet_x_32gf"""
        model_ft = regnet_x_32gf(weights=None)
        checkpoints = torch.load('pretrained_weights/regnet_x_32gf-6eb8fdc6.pth')
        model_ft.load_state_dict(checkpoints)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.fc.in_features
        model_ft.fc = nn.Linear(num_ftrs, num_classes)
        input_size = 224

    if model_name == "swin":
        """swin_b"""
        model_ft = swin_b(weights=None)
        checkpoints = torch.load('pretrained_weights/swin_b-68c6b09e.pth')
        model_ft.load_state_dict(checkpoints)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.head.in_features
        model_ft.head = nn.Linear(num_ftrs, num_classes)
        input_size = 224

    if model_name == "maxvit":
        """maxvit"""
        model_ft = maxvit_t(weights=None)
        checkpoints = torch.load('pretrained_weights/maxvit_t-bc5ab103.pth')
        model_ft.load_state_dict(checkpoints)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.classifier[5].in_features
        model_ft.classifier[5] = nn.Linear(num_ftrs, num_classes)
        
    return model_ft, input_size
        
def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False

In [5]:
class SlideLevelClassifier(nn.Module):
    def __init__(self, num_features, num_classes):
        super(SlideLevelClassifier, self).__init__()
        self.fc1 = nn.Linear(num_features, 512)
        self.fc2 = nn.Linear(512, num_classes)
    
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x
    
def pool_features(features, method="average"):
    if method == "average":
        return torch.mean(features, dim=0)
    elif method == "max":
        return torch.max(features, dim=0)[0]
    else:
        raise valueError("Invalid pooling method")
        
class BiopsyDataset(Dataset):
    def __init__(self, biopsy_feature_paths, labels):
        self.biopsy_feature_paths = biopsy_feature_paths
        self.labels = labels
    
    def __len__(self):
        return len(self.biopsy_feature_paths)
    
    def __getitem__(self, idx):
       # Load precomputed features for all slides of the current biopsy
        features_list = [torch.load(slide_feature_path) for slide_feature_path in self.biopsy_feature_paths[idx]]

        # Instead of stacking, we will average the features for each slide
        # This assumes that the feature tensors have the shape [num_tiles, num_features]
        # and that you want to average across the num_tiles dimension
        slide_features = torch.stack([features.mean(dim=0) for features in features_list])

        # Now we can stack since all slide feature tensors will have the same shape
        # Then we take the mean across slides to get a single feature vector for the biopsy
        biopsy_feature_vector = slide_features.mean(dim=0)

        return biopsy_feature_vector, self.labels[idx]


In [6]:
import os
from pathlib import Path
from openslide import OpenSlide
from PIL import Image
import matplotlib.pyplot as plt
import h5py
import torch

In [7]:
brca_dir = Path().home() / 'datasets' / 'brca-psj-path'
ndpi_dir = brca_dir / 'ndpi'
clam_train_dir = brca_dir / 'contest-phase-2' / 'clam-preprocessing-train'
features_pt_dir = clam_train_dir / 'resnet50-features'/ 'pt_files'

In [8]:
pt_features = torch.load(features_pt_dir / f'{slide_id}.pt')

NameError: name 'slide_id' is not defined

In [8]:
def get_feature_path(slide_id):
    return f'{features_pt_dir}/{slide_id}.pt'

In [9]:
df_train = pd.read_csv("/home/ngsci/project/Armin/metadata/train.csv")
df_val = pd.read_csv("/home/ngsci/project/Armin/metadata/valid.csv")
grouped_train = df_train.groupby("biopsy_id")["slide_id"].apply(lambda x: [get_feature_path(slide) for slide in x])
grouped_val = df_val.groupby("biopsy_id")["slide_id"].apply(lambda x: [get_feature_path(slide) for slide in x])
biopsy_feature_paths_train = grouped_train.tolist()
biopsy_feature_paths_valid = grouped_val.tolist()
labels_train = df_train.groupby("biopsy_id").agg({
    "stage": "first"}).reset_index().stage.tolist()
labels_valid = df_val.groupby("biopsy_id").agg({
    "stage": "first"}).reset_index().stage.tolist()

In [10]:
train_dataset = BiopsyDataset(biopsy_feature_paths_train, labels_train)
train_dataloader = DataLoader(train_dataset, batch_size=4, shuffle=True)
val_dataset = BiopsyDataset(biopsy_feature_paths_valid, labels_valid)
val_dataloader = DataLoader(val_dataset, batch_size=4, shuffle=True)
model = SlideLevelClassifier(num_features=1024, num_classes=2)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

num_epochs = 20

In [21]:
from tqdm import tqdm
best_val_loss = 1000000
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for batch_features, batch_labels in tqdm(train_dataloader):
        optimizer.zero_grad()
        outputs = model(batch_features)
        loss = criterion(outputs, batch_labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    
    training_loss = running_loss / len(train_dataloader)
    model.eval()
    running_val_loss = 0.0
    with torch.no_grad():
        for val_features, val_labels in val_dataloader:
            val_outputs = model(val_features)
            val_loss = criterion(val_outputs, val_labels)
            running_val_loss += val_loss.item()
    validation_loss = running_val_loss / len(val_dataloader)
    
    if validation_loss < best_val_loss:
        best_val_loss = validation_loss
        torch.save(model.state_dict(), "/home/ngsci/project/Armin/saved_models/breast_cancer_attention.pth")
    
    print(f'Epoch [{epoch+1}/{num_epochs}], Training Loss: {training_loss:.4f}, Validation Loss: {validation_loss:.4f}')

100%|██████████| 82/82 [1:02:35<00:00, 45.79s/it]


Epoch [1/20], Training Loss: 1.1015, Validation Loss: 0.8560


100%|██████████| 82/82 [1:04:12<00:00, 46.98s/it] 


Epoch [2/20], Training Loss: 0.7755, Validation Loss: 0.7032


100%|██████████| 82/82 [1:03:28<00:00, 46.44s/it]


...

Epoch [18/20], Training Loss: 0.5825, Validation Loss: 0.7067


100%|██████████| 82/82 [54:31<00:00, 39.89s/it]  


Epoch [19/20], Training Loss: 0.5835, Validation Loss: 0.7137


100%|██████████| 82/82 [55:16<00:00, 40.44s/it]  


Epoch [20/20], Training Loss: 0.5722, Validation Loss: 0.7459


In [24]:
torch.save(model.state_dict(), "/home/ngsci/project/Armin/saved_models/breast_cancer_attention_end.pth")

In [11]:
df_test = pd.read_csv("/home/ngsci/project/Armin/metadata/test.csv")
grouped_test = df_test.groupby("biopsy_id")["slide_id"].apply(lambda x: [get_feature_path(slide) for slide in x])
biopsy_feature_paths_test = grouped_test.tolist()
labels_test = df_test.groupby("biopsy_id").agg({
    "stage": "first"}).reset_index().stage.tolist()

In [12]:
test_dataset = BiopsyDataset(biopsy_feature_paths_test, labels_test)
test_dataloader = DataLoader(test_dataset, batch_size=1, shuffle=False)

In [36]:
df_test.head()

,Unnamed: 0,biopsy_id,stage,race,slide_id
0,21,00c619a9-7f7d-4d18-bd87-7c2bd482fc05,0,1,1e0502bb-e0c9-45a8-99d9-212f945de9c8
1,22,00c619a9-7f7d-4d18-bd87-7c2bd482fc05,0,1,d6cf5a07-db91-45af-83b5-14531f50c42f
2,128,02b059c2-5445-453b-9ba6-636812b5089d,0,4,82467cf2-671b-4e19-910d-eb0a49570f7e
3,129,02b059c2-5445-453b-9ba6-636812b5089d,0,4,b13358ab-487b-4352-a4c7-810b170b7b64
4,366,085c5190-e25e-4443-b53f-483121f711ca,0,1,376d279c-03d1-437d-beaf-6fd58d9334db


In [15]:
model = SlideLevelClassifier(num_features=1024, num_classes=2)
model.load_state_dict(torch.load("/home/ngsci/project/Armin/saved_models/breast_cancer_attention_end.pth"))

RuntimeError: Error(s) in loading state_dict for SlideLevelClassifier:
	size mismatch for fc2.weight: copying a param with shape torch.Size([4, 512]) from checkpoint, the shape in current model is torch.Size([2, 512]).
	size mismatch for fc2.bias: copying a param with shape torch.Size([4]) from checkpoint, the shape in current model is torch.Size([2]).

In [16]:
# Storage for the results
results = []

# Iterate over the test data
for i, (inputs, _) in enumerate(test_dataloader):
    with torch.no_grad():
        outputs = model(inputs)
        # Assuming you're doing classification and you want to get the class with the maximum score
        _, preds = torch.max(outputs, 1)
        results.extend(preds.cpu().numpy())

# Assuming biopsy_ids are unique and their order in labels_test corresponds to the order in the DataLoader
biopsy_ids = df_test['biopsy_id'].unique()



In [18]:
df_test.head()

,Unnamed: 0,biopsy_id,stage,race,slide_id
0,21,00c619a9-7f7d-4d18-bd87-7c2bd482fc05,0,1,1e0502bb-e0c9-45a8-99d9-212f945de9c8
1,22,00c619a9-7f7d-4d18-bd87-7c2bd482fc05,0,1,d6cf5a07-db91-45af-83b5-14531f50c42f
2,128,02b059c2-5445-453b-9ba6-636812b5089d,0,4,82467cf2-671b-4e19-910d-eb0a49570f7e
3,129,02b059c2-5445-453b-9ba6-636812b5089d,0,4,b13358ab-487b-4352-a4c7-810b170b7b64
4,366,085c5190-e25e-4443-b53f-483121f711ca,0,1,376d279c-03d1-437d-beaf-6fd58d9334db


In [15]:
# Save to CSV
result_df = pd.DataFrame({
    'biopsy_id': biopsy_ids,
    'pred_stage': results,
    'actual_stage': df_test.groupby("biopsy_id").agg({"stage": "first"}).reset_index().stage.tolist(),
    "race": df_test.groupby("biopsy_id").agg({"race": "first"}).reset_index().race.tolist(),
    'slide_id': df_test.groupby("biopsy_id").agg({"stage": "first", 'slide_id': "first"}).reset_index().slide_id.tolist()
})

result_df.to_csv('predictions/predictions_attention.csv', index=False)

NameError: name 'biopsy_ids' is not defined

In [37]:
data_dir = os.path.join("/", "home", "ngsci", "project", "Armin", "metadata")

num_classes = 2
batch_size = 32
num_epochs = 20
feature_extract = False
num_gpus = [i for i in range(torch.cuda.device_count())]
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

if len(num_gpus) > 1:
    print("We are using {} gpus".format(len(num_gpus)))
    os.environ["CUDA_VISIBLE_DEVICES"] = ','.join(str(x) for x in num_gpus)

In [38]:
data_dir = os.path.join("/", "home", "ngsci", "project", "Armin", "metadata2")
num_classes = 3
batch_size = 32
num_epochs = 20
feature_extract = False
num_gpus = [i for i in range(torch.cuda.device_count())]
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
if len(num_gpus) > 1:
    print("We are using {} gpus".format(len(num_gpus)))
    os.environ["CUDA_VISISBLE_DEVICES"] = ','.join(str(x) for x in num_gpus)

In [47]:
# ["resnet_18", "resnet50", "wide_resnet101", "resnet152", "efficientnet", "convnext", "vgg", "regnet", "swin", "maxvit"]
for model_name in ["resnet_18", "resnet50", "wide_resnet101", "resnet152", "efficientnet", "convnext", "vgg", "regnet", "swin", "maxvit"]:
    model_ft, input_size = initialize_model(model_name, num_classes, feature_extract, use_pretrained=True)
    data_transforms = {
        "train": transforms.Compose([
            transforms.Resize(input_size),
            transforms.RandomResizedCrop(input_size),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ]),
        "valid": transforms.Compose([
            transforms.Resize(input_size),
            transforms.CenterCrop(input_size),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ]),
        "test": transforms.Compose([
            transforms.Resize(input_size),
            transforms.CenterCrop(input_size),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ])
    }
    image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x), data_transforms[x]) for x in ["train", "valid", "test"]}
    dataloader_dict = {x[0]: torch.utils.data.DataLoader(image_datasets[x[0]], batch_size=batch_size, shuffle=x[1], num_workers=4) for x in [("train", True), ("valid", False), ("test", False)]}
    num_gpus = [i for i in range(torch.cuda.device_count())]
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    if len(num_gpus) > 1:
        os.environ["CUDA_VISIBLE_DEVICES"] = ','.join(str(x) for x in num_gpus)
        model_ft = torch.nn.DataParallel(model_ft, device_ids=num_gpus)
        model_ft = model_ft.module
    model_ft = model_ft.to(device)
    
    params_to_update = model_ft.parameters()
    if feature_extract:
        params_to_update = []
        for name, param in model_ft.named_parameters():
            if param.requires_grad == True:
                params_to_update.append(param)
    
    model_lr_map = {
        "resnet_18": 1e-5,
        "resnet50": 1e-4, 
        "wide_resnet101": 1e-4, 
        "resnet152": 1e-4, 
        "efficientnet": 1e-4, 
        "convnext": 1e-4, 
        "vgg": 1e-4, 
        "regnet": 1e-4, 
        "swin": 1e-4, 
        "maxvit": 1e-4
    }
    lr_ = model_lr_map[model_name]
    
    optimizer_ft = optim.AdamW(params_to_update, lr=lr_)
    criterion = nn.CrossEntropyLoss()
    model_ft = train_model(model_ft, model_name, dataloader_dict, criterion, optimizer_ft, num_classes, num_epochs, False, lr_, df_train)

Epoch 1/20
----------
train Loss: 1.2084 Acc: 0.3402
valid Loss: 0.9554 Acc: 0.5843

Epoch 2/20
----------
train Loss: 0.8233 Acc: 0.6513
valid Loss: 0.8522 Acc: 0.6050

Epoch 3/20
----------
train Loss: 0.7624 Acc: 0.6595
valid Loss: 0.8319 Acc: 0.5977

Epoch 4/20
----------
train Loss: 0.7356 Acc: 0.6742
valid Loss: 0.8100 Acc: 0.6060

Epoch 5/20
----------
train Loss: 0.7258 Acc: 0.6738
valid Loss: 0.8281 Acc: 0.5977

Epoch 6/20
----------
train Loss: 0.7171 Acc: 0.6840
valid Loss: 0.8165 Acc: 0.6101

Epoch 7/20
----------
train Loss: 0.7111 Acc: 0.6812
valid Loss: 0.8015 Acc: 0.6101

Epoch 8/20
----------
train Loss: 0.6926 Acc: 0.6979
valid Loss: 0.7980 Acc: 0.6091

Epoch 9/20
----------
train Loss: 0.6831 Acc: 0.6983
valid Loss: 0.8043 Acc: 0.6081

Epoch 10/20
----------
train Loss: 0.6742 Acc: 0.7052
valid Loss: 0.8332 Acc: 0.6060

Epoch 11/20
----------
train Loss: 0.6644 Acc: 0.7102
valid Loss: 0.8264 Acc: 0.6122

Epoch 12/20
----------
train Loss: 0.6565 Acc: 0.7152
valid Los

ValueError: Caught ValueError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/opt/venv/default/lib/python3.10/site-packages/torch/utils/data/_utils/worker.py", line 308, in _worker_loop
    data = fetcher.fetch(index)
  File "/opt/venv/default/lib/python3.10/site-packages/torch/utils/data/_utils/fetch.py", line 51, in fetch
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/opt/venv/default/lib/python3.10/site-packages/torch/utils/data/_utils/fetch.py", line 51, in <listcomp>
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/opt/venv/default/lib/python3.10/site-packages/torchvision/datasets/folder.py", line 231, in __getitem__
    sample = self.transform(sample)
  File "/opt/venv/default/lib/python3.10/site-packages/torchvision/transforms/transforms.py", line 95, in __call__
    img = t(img)
  File "/opt/venv/default/lib/python3.10/site-packages/torch/nn/modules/module.py", line 1501, in _call_impl
    

In [13]:
from PIL import Image
from tqdm import tqdm
import csv
import pandas as pd

def run_inference_image(path, model):
    model.eval()
    biopsy_id = os.path.basename(path).split('.')[0]
    image = Image.open(path)
    
    transform_data = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.CenterCrop((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
    img_t = transform_data(image)
    img_t = img_t.float().unsqueeze(0).to(device)
    with torch.no_grad():
        output = model(img_t)
    
    prediction = output.squeeze(0).softmax(0)
    class_id = prediction.argmax().item()
    return biopsy_id, class_id

def run_inference(paths, model):
    predictions = []
    for index in tqdm(range(len(paths)), desc="Evaluation Progress"):
        predictions.append(run_inference_image(paths[index], model))
    pred_dict = {biopsy: class_id for biopsy, class_id in predictions}
    return pred_dict

def save_predictions(pred_dict, model_name):
    frame = pd.DataFrame()
    frame["slide_id"] = list(pred_dict.keys())
    frame["pred_stage"] = list(pred_dict.values())
    frame.to_csv("predictions/predictions_{}_{}.csv".format(model_name, batch_size))
    

In [14]:
def build_model(model_name, num_classes=2):
    if model_name == "resnet18":
        lr = 1e-5
        model_ft = resnet18(weights=None)
        checkpoints = torch.load("saved_models/breast_cancer_resnet_18_1e-05_20_2.pt")
        num_ftrs = model_ft.fc.in_features
        model_ft.fc = nn.Linear(num_ftrs, num_classes)
        checkpoints = {k: v for k, v in checkpoints.items() if "fc" not in k}
        model_ft.load_state_dict(checkpoints, strict=False)
        return model_ft
    
    if model_name == "resnet50":
        lr = 1e-4
        model_ft = resnet50(weights=None)
        checkpoints = torch.load('saved_models/breast_cancer_resnet50_0.0001_20_2.pt')
        num_ftrs = model_ft.fc.in_features
        model_ft.fc = nn.Linear(num_ftrs, num_classes)
        checkpoints = {k: v for k, v in checkpoints.items() if "fc" not in k}
        model_ft.load_state_dict(checkpoints, strict=False)
        return model_ft

    if model_name == "resnet152":
        lr = 1e-5
        model_ft = resnet152(weights=None)
        checkpoints = torch.load('saved_models/breast_cancer_resnet152_0.0001_20_2.pt')
        num_ftrs = model_ft.fc.in_features
        model_ft.fc = nn.Linear(num_ftrs, num_classes)
        checkpoints = {k: v for k, v in checkpoints.items() if "fc" not in k}
        model_ft.load_state_dict(checkpoints, strict=False)
        return model_ft
    
    if model_name == "wide_resnet101":
        lr = 1e-4
        model_ft = wide_resnet101_2(weights=None)
        checkpoints = torch.load('saved_models/breast_cancer_wide_resnet101_0.0001_20_2.pt')
        num_ftrs = model_ft.fc.in_features
        model_ft.fc = nn.Linear(num_ftrs, num_classes)
        checkpoints = {k: v for k, v in checkpoints.items() if "fc" not in k}
        model_ft.load_state_dict(checkpoints, strict=False)
        return model_ft

    if model_name == "vgg":
        lr = 1e-4
        model_ft = vgg19_bn(weights=None)
        checkpoints = torch.load('saved_models/breast_cancer_vgg_0.0001_20_2.pt')
        num_ftrs = model_ft.classifier[6].in_features
        model_ft.classifier[6] = nn.Linear(num_ftrs, num_classes)  # Set to 3 for your case
        # This part of the code is important: we are only loading the state_dict for layers
        # that match in size, we are ignoring ('skipping') the classifier.6 layer since it doesn't match.
        checkpoints = {k: v for k, v in checkpoints.items() if k not in ['classifier.6.weight', 'classifier.6.bias']}
        model_ft.load_state_dict(checkpoints, strict=False)
        return model_ft
    
    if model_name == "efficientnet":
        lr = 4e-4
        model_ft = efficientnet_v2_m(weights=None)
        checkpoints = torch.load('saved_models/breast_cancer_efficientnet_0.0001_20_2.pt')
        num_ftrs = model_ft.classifier[1].in_features
        model_ft.classifier = nn.Linear(num_ftrs, num_classes)
        model_ft.load_state_dict(checkpoints, strict=False)
        return model_ft
    
    if model_name == "convnext":
        lr = 1e-5
        model_ft = convnext_base(weights=None)
        checkpoints = torch.load('saved_models/breast_cancer_convnext_0.0001_20_2.pt')
        num_ftrs = model_ft.classifier[2].in_features
        # Redefine the last layer to have 3 outputs instead of 2
        model_ft.classifier[2] = nn.Linear(num_ftrs, num_classes) 
        # Exclude the last layer's weights and bias when loading the checkpoint
        checkpoints = {k: v for k, v in checkpoints.items() if k not in ['classifier.2.weight', 'classifier.2.bias']}
        # Load the model with strict=False to avoid size mismatch errors
        model_ft.load_state_dict(checkpoints, strict=False)
        return model_ft

    if model_name == "regnet":
        lr = 1e-5
        model_ft = regnet_x_32gf(weights=None)
        checkpoints = torch.load('saved_models/breast_cancer_regnet_0.0001_20_2.pt')
        num_ftrs = model_ft.fc.in_features
        model_ft.fc = nn.Linear(num_ftrs, num_classes)  # Redefine the fully connected layer with the correct number of classes
        # Exclude the fc.weight and fc.bias when loading the checkpoint
        checkpoints = {k: v for k, v in checkpoints.items() if k not in ['fc.weight', 'fc.bias']}
        # Load the model with strict=False to avoid size mismatch errors
        model_ft.load_state_dict(checkpoints, strict=False)
        return model_ft

    if model_name == "swin":
        lr = 1e-5
        model_ft = swin_b(weights=None)
        checkpoints = torch.load('saved_models/breast_cancer_swin_0.0001_20_3.pt')
        num_ftrs = model_ft.head.in_features
        model_ft.head = nn.Linear(num_ftrs, num_classes)
        model_ft.load_state_dict(checkpoints)
        return model_ft

    # if model_name == "maxvit":
    #     lr = 1e-4
    #     model_ft = maxvit_t(weights=None)
    #     checkpoints = torch.load('best_final_weights/breast_cancer_{}_{}_{}_{}.pt'.format(model_name, batch_size, num_epochs, lr))
    #     num_ftrs = model_ft.classifier[5].in_features
    #     model_ft.classifier[5] = nn.Linear(num_ftrs, num_classes)
    #     model_ft.load_state_dict(checkpoints)
    #     return model_ft

In [15]:
def get_biopsy_id(df, slide_id_value):
    # Step 2: Filter rows with the given slide_id
    result = df[df['slide_id'] == slide_id_value]
    
    # Step 3: Extract the corresponding biopsy_id
    if not result.empty:
        return result['biopsy_id'].iloc[0]
    else:
        return None

In [16]:
def get_race(df, slide_id_value):
    result = df[df["slide_id"] == slide_id_value]
    if not result.empty:
        return result["race"].iloc[0]
    else:
        return None

In [17]:
# Function to group races
def group_race(race):
    if (race in ['2', '3', '4', '8', '9']) or (race in [2, 3, 4, 5, 8, 9]):
        return 'grouped_race'
    else:
        return race

In [14]:
df = pd.read_csv("/home/ngsci/project/Armin/metadata/data.csv")
for model_name in ["resnet18", "resnet50", "wide_resnet101", "resnet152", "vgg", "efficientnet", "convnext", "regnet"]:
    model = build_model(model_name)
    model = to_device(model)
    data_dir = "/home/ngsci/project/Armin/metadata"
    test_dir = os.path.join(data_dir, "test")
    test_image_paths_0 = [os.path.join(test_dir, "0", fname) for fname in os.listdir(os.path.join(test_dir, "0"))]
    test_image_paths_1 = [os.path.join(test_dir, "1", fname) for fname in os.listdir(os.path.join(test_dir, "1"))]
    # Create or overwrite the CSV file
    with open('predictions/predictions_{}.csv'.format(model_name), 'w', newline='') as csvfile:
        fieldnames = ['biopsy_id', 'slide_id', 'race', 'pred_stage', 'actual_stage']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()

        for path in tqdm(test_image_paths_0, desc="Predicting for label 0"):
            prediction = run_inference_image(path, model)
            biopsy_id = get_biopsy_id(df, prediction[0])
            race = group_race(get_race(df, prediction[0]))
            writer.writerow({'biopsy_id': biopsy_id, 'slide_id': prediction[0], 'race': race, 'pred_stage': prediction[1], 'actual_stage': 0})

        for path in tqdm(test_image_paths_1, desc="Predicting for label 1"):
            prediction = run_inference_image(path, model)
            biopsy_id = get_biopsy_id(df, prediction[0])
            race = group_race(get_race(df, prediction[0]))
            writer.writerow({'biopsy_id': biopsy_id, 'slide_id': prediction[0], 'race': race, 'pred_stage': prediction[1], 'actual_stage': 1})
    # pred_dict = run_inference(test_image_paths, model)
    # save_predictions(pred_dict, model_name)

RuntimeError: Attempting to deserialize object on a CUDA device but torch.cuda.is_available() is False. If you are running on a CPU-only machine, please use torch.load with map_location=torch.device('cpu') to map your storages to the CPU.

In [18]:
from sklearn.utils import resample

# Function to perform bootstrapping and calculate metrics
def bootstrap_metrics(actuals, predictions, n_iterations=50, sample_size=None):
    bootstrap_accuracy = []
    bootstrap_precision = []
    bootstrap_recall = []
    bootstrap_f1 = []
    
    if sample_size is None:
        sample_size = len(actuals)

    for i in range(n_iterations):
        # Create a random subsample of the data with replacement
        actuals_sample, predictions_sample = resample(actuals, predictions, n_samples=sample_size)
        
        # Calculate metrics on this subsample
        accuracy = accuracy_score(actuals_sample, predictions_sample) * 100
        precision = precision_score(actuals_sample, predictions_sample, average="weighted")
        recall = recall_score(actuals_sample, predictions_sample, average="weighted")
        f1 = f1_score(actuals_sample, predictions_sample, average='weighted')
        
        # Store the metrics
        bootstrap_accuracy.append(accuracy)
        bootstrap_precision.append(precision)
        bootstrap_recall.append(recall)
        bootstrap_f1.append(f1)
    
    # Convert lists to numpy arrays
    bootstrap_accuracy = np.array(bootstrap_accuracy)
    bootstrap_precision = np.array(bootstrap_precision)
    bootstrap_recall = np.array(bootstrap_recall)
    bootstrap_f1 = np.array(bootstrap_f1)

    # Calculate the mean and standard deviation of the bootstrap samples
    metrics = {
        'accuracy': {'mean': np.mean(bootstrap_accuracy), 'std': np.std(bootstrap_accuracy)},
        'precision': {'mean': np.mean(bootstrap_precision), 'std': np.std(bootstrap_precision)},
        'recall': {'mean': np.mean(bootstrap_recall), 'std': np.std(bootstrap_recall)},
        'f1': {'mean': np.mean(bootstrap_f1), 'std': np.std(bootstrap_f1)}
    }

    return metrics

In [20]:
from PIL import Image
from tqdm import tqdm
import csv
import pandas as pd
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score


slide_to_race = {}
with open('metadata/data.csv', 'r') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        slide_to_race[row['slide_id']] = group_race(row['race'])

predictions = {}
actuals = {}
biopsy_ids = {}
races = set()

for model_name in ["resnet18", "resnet50", "wide_resnet101", "resnet152", "vgg", "efficientnet", "convnext", "regnet"]:
    with open('predictions/predictions_{}.csv'.format(model_name), 'r') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            slide_id = row['slide_id']
            if slide_id in slide_to_race.keys():
                race = slide_to_race[slide_id]
                races.add(race)
                if race not in predictions:
                    predictions[race] = []
                    actuals[race] = []
                predictions[race].append(int(row['pred_stage']))
                actuals[race].append(int(row['actual_stage']))
    
    for race in races:
        metrics = bootstrap_metrics(actuals[race], predictions[race])
        print(f"Model Name: {model_name}")
        print(f"Race: {race}")
        print(f"Bootstrap Accuracy: {metrics['accuracy']['mean']:.2f}% ± {metrics['accuracy']['std']:.2f}")
        print(f"Bootstrap Precision: {metrics['precision']['mean']:.2f} ± {metrics['precision']['std']:.2f}")
        print(f"Bootstrap Recall: {metrics['recall']['mean']:.2f} ± {metrics['recall']['std']:.2f}")
        print(f"Bootstrap Weighted F1 Score: {metrics['f1']['mean']:.2f} ± {metrics['f1']['std']:.2f}")
        print("-----------------------------")
        

Model Name: resnet18
Race: 1
Bootstrap Accuracy: 70.62% ± 2.83
Bootstrap Precision: 0.76 ± 0.03
Bootstrap Recall: 0.71 ± 0.03
Bootstrap Weighted F1 Score: 0.73 ± 0.03
-----------------------------


/opt/venv/default/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Model Name: resnet18
Race: grouped_race
Bootstrap Accuracy: 55.27% ± 6.33
Bootstrap Precision: 0.66 ± 0.10
Bootstrap Recall: 0.55 ± 0.06
Bootstrap Weighted F1 Score: 0.46 ± 0.08
-----------------------------
Model Name: resnet50
Race: 1
Bootstrap Accuracy: 67.37% ± 1.75
Bootstrap Precision: 0.77 ± 0.02
Bootstrap Recall: 0.67 ± 0.02
Bootstrap Weighted F1 Score: 0.71 ± 0.02
-----------------------------
Model Name: resnet50
Race: grouped_race
Bootstrap Accuracy: 61.95% ± 4.31
Bootstrap Precision: 0.74 ± 0.04
Bootstrap Recall: 0.62 ± 0.04
Bootstrap Weighted F1 Score: 0.57 ± 0.05
-----------------------------
Model Name: wide_resnet101
Race: 1
Bootstrap Accuracy: 58.31% ± 1.66
Bootstrap Precision: 0.76 ± 0.02
Bootstrap Recall: 0.58 ± 0.02
Bootstrap Weighted F1 Score: 0.64 ± 0.02
-----------------------------
Model Name: wide_resnet101
Race: grouped_race
Bootstrap Accuracy: 56.72% ± 4.24
Bootstrap Precision: 0.57 ± 0.04
Bootstrap Recall: 0.57 ± 0.04
Bootstrap Weighted F1 Score: 0.57 ± 0.04


In [21]:
resnet18 = pd.read_csv("predictions/predictions_resnet18.csv")
resnet50 = pd.read_csv("predictions/predictions_resnet50.csv")
resnet152 = pd.read_csv("predictions/predictions_resnet152.csv")
efficientnet = pd.read_csv("predictions/predictions_efficientnet.csv")
convnext = pd.read_csv("predictions/predictions_convnext.csv")
wide_resnet = pd.read_csv("predictions/predictions_wide_resnet101.csv")
vgg = pd.read_csv("predictions/predictions_vgg.csv")
regnet = pd.read_csv("predictions/predictions_regnet.csv")

slide_ids = resnet50.slide_id.tolist()
actual_stages = resnet50.actual_stage.tolist()
biopsy_ids = resnet50.biopsy_id.tolist()
races = resnet50.race.tolist()
rest50_preds = resnet50.pred_stage.tolist()
efficientnet_preds = efficientnet.pred_stage.tolist()
convnext_preds = convnext.pred_stage.tolist()
wide_resnet_preds = wide_resnet.pred_stage.tolist()
vgg_preds = vgg.pred_stage.tolist()
regnet_preds = regnet.pred_stage.tolist()

# Using only models that have loss < 1
avg_preds = [round((r50 + effnet + convnext_pred + wide_resnet_pred + vgg_pred + regnet_pred )/6)
             for r50, effnet, convnext_pred, wide_resnet_pred, vgg_pred, regnet_pred in 
             zip(rest50_preds, efficientnet_preds, convnext_preds, wide_resnet_preds, vgg_preds, regnet_preds)]

final_frame = pd.DataFrame()
final_frame['slide_id'] = slide_ids
final_frame["biopsy_id"] = biopsy_ids
final_frame['pred_stage'] = avg_preds
final_frame["actual_stage"] = actual_stages
final_frame["race"] = races

aggregated_df = final_frame.groupby('biopsy_id').agg({
    'pred_stage': 'mean',
    'slide_id': 'first',
    'actual_stage': 'first',
    'race': 'first'
}).reset_index()

# Round the mean value to the nearest integer
aggregated_df['pred_stage'] = aggregated_df['pred_stage'].round().astype(int)

# print(aggregated_df.head())
aggregated_df.to_csv('predictions/predictions_final.csv', index=False)
# final_frame.head()

In [22]:
from PIL import Image
from tqdm import tqdm
import csv
import pandas as pd
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score


slide_to_race = {}
with open('metadata/data.csv', 'r') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        slide_to_race[row['slide_id']] = group_race(row['race'])

predictions = {}
actuals = {}
biopsy_ids = {}
races = set()

with open('predictions/predictions_final.csv', 'r') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        slide_id = row['slide_id']
        if slide_id in slide_to_race.keys():
            race = slide_to_race[slide_id]
            races.add(race)
            if race not in predictions:
                predictions[race] = []
                actuals[race] = []
            predictions[race].append(int(row['pred_stage']))
            actuals[race].append(int(row['actual_stage']))

for race in races:
    metrics = bootstrap_metrics(actuals[race], predictions[race])
    print(f"Model Name: {model_name}")
    print(f"Race: {race}")
    print(f"Bootstrap Accuracy: {metrics['accuracy']['mean']:.2f}% ± {metrics['accuracy']['std']:.2f}")
    print(f"Bootstrap Precision: {metrics['precision']['mean']:.2f} ± {metrics['precision']['std']:.2f}")
    print(f"Bootstrap Recall: {metrics['recall']['mean']:.2f} ± {metrics['recall']['std']:.2f}")
    print(f"Bootstrap Weighted F1 Score: {metrics['f1']['mean']:.2f} ± {metrics['f1']['std']:.2f}")
    print("-----------------------------")

Model Name: regnet
Race: 1
Bootstrap Accuracy: 62.88% ± 8.76
Bootstrap Precision: 0.67 ± 0.09
Bootstrap Recall: 0.63 ± 0.09
Bootstrap Weighted F1 Score: 0.62 ± 0.09
-----------------------------
Model Name: regnet
Race: grouped_race
Bootstrap Accuracy: 29.56% ± 13.26
Bootstrap Precision: 0.79 ± 0.34
Bootstrap Recall: 0.30 ± 0.13
Bootstrap Weighted F1 Score: 0.34 ± 0.18
-----------------------------


/opt/venv/default/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/venv/default/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/venv/default/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/venv/default/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWar

In [23]:
from PIL import Image
from tqdm import tqdm
import csv
import pandas as pd
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Function to group races
def group_race(race):
    if race in ['2', '3', '4', '8', '9']:
        return 'grouped_race'
    else:
        return race

slide_to_race = {}
with open('metadata/data.csv', 'r') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        slide_to_race[row['slide_id']] = group_race(row['race'])

predictions = {}
actuals = {}
races = set()
with open('predictions/predictions_attention.csv') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            slide_id = row['slide_id']
            if slide_id in slide_to_race.keys():
                race = slide_to_race[slide_id]
                races.add(race)
                if race not in predictions:
                    predictions[race] = []
                    actuals[race] = []
                predictions[race].append(int(row['pred_stage']))
                actuals[race].append(int(row['actual_stage']))

for race in races:
    metrics = bootstrap_metrics(actuals[race], predictions[race])
    print(f"Model Name: {model_name}")
    print(f"Race: {race}")
    print(f"Bootstrap Accuracy: {metrics['accuracy']['mean']:.2f}% ± {metrics['accuracy']['std']:.2f}")
    print(f"Bootstrap Precision: {metrics['precision']['mean']:.2f} ± {metrics['precision']['std']:.2f}")
    print(f"Bootstrap Recall: {metrics['recall']['mean']:.2f} ± {metrics['recall']['std']:.2f}")
    print(f"Bootstrap Weighted F1 Score: {metrics['f1']['mean']:.2f} ± {metrics['f1']['std']:.2f}")
    print("-----------------------------")

Model Name: regnet
Race: 1
Bootstrap Accuracy: 65.44% ± 8.18
Bootstrap Precision: 0.68 ± 0.08
Bootstrap Recall: 0.65 ± 0.08
Bootstrap Weighted F1 Score: 0.65 ± 0.08
-----------------------------
Model Name: regnet
Race: grouped_race
Bootstrap Accuracy: 55.11% ± 16.47
Bootstrap Precision: 0.77 ± 0.24
Bootstrap Recall: 0.55 ± 0.16
Bootstrap Weighted F1 Score: 0.63 ± 0.17
-----------------------------


/opt/venv/default/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/venv/default/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/venv/default/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/venv/default/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWar

In [44]:
import csv
df = pd.read_csv("/home/ngsci/project/Armin/metadata/data.csv")
for model_name in ["resnet18", "resnet50", "wide_resnet101", "resnet152", "vgg", "efficientnet", "convnext", "regnet"]:
    model = build_model(model_name, 3)
    model = to_device(model)
    data_dir = "/home/ngsci/project/Armin/metadata2"
    test_dir = os.path.join(data_dir, "test")
    test_image_paths_0 = [os.path.join(test_dir, "0", fname) for fname in os.listdir(os.path.join(test_dir, "0"))]
    test_image_paths_1 = [os.path.join(test_dir, "1", fname) for fname in os.listdir(os.path.join(test_dir, "1"))]
    test_image_paths_2 = [os.path.join(test_dir, "2", fname) for fname in os.listdir(os.path.join(test_dir, "2"))]
    # Create or overwrite the CSV file
    with open('predictions/predictions_{}_{}_train.csv'.format(model_name, 3), 'w', newline='') as csvfile:
        fieldnames = ['slide_id', 'pred_stage', 'actual_stage', "biopsy_id", "race"]
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()

        for path in tqdm(test_image_paths_0, desc="Predicting for label 0"):
            prediction = run_inference_image(path, model)
            biopsy_id = get_biopsy_id(df, prediction[0])
            race = group_race(get_race(df, prediction[0]))
            writer.writerow({"biopsy_id": biopsy_id, 'slide_id': prediction[0], "race": race, 'pred_stage': prediction[1], 'actual_stage': 0})

        for path in tqdm(test_image_paths_1, desc="Predicting for label 1"):
            prediction = run_inference_image(path, model)
            biopsy_id = get_biopsy_id(df, prediction[0])
            race = group_race(get_race(df, prediction[0]))
            writer.writerow({"biopsy_id": biopsy_id, 'slide_id': prediction[0], "race": race, 'pred_stage': prediction[1], 'actual_stage': 1})
        
        for path in tqdm(test_image_paths_2, desc="Predicting for label 2"):
            prediction = run_inference_image(path, model)
            biopsy_id = get_biopsy_id(df, prediction[0])
            race = group_race(get_race(df, prediction[0]))
            writer.writerow({"biopsy_id": biopsy_id, 'slide_id': prediction[0], "race": race, 'pred_stage': prediction[1], 'actual_stage': 2})

RuntimeError: Attempting to deserialize object on a CUDA device but torch.cuda.is_available() is False. If you are running on a CPU-only machine, please use torch.load with map_location=torch.device('cpu') to map your storages to the CPU.

In [24]:
from PIL import Image
from tqdm import tqdm
import csv
import pandas as pd
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Function to group races
def group_race(race):
    if race in ['2', '3', '4', '5', '8', '9']:
        return 'grouped_race'
    else:
        return race

slide_to_race = {}
with open('metadata/data.csv', 'r') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        slide_to_race[row['slide_id']] = group_race(row['race'])

predictions = {}
actuals = {}
races = set()
for model_name in ["resnet18", "resnet50", "wide_resnet101", "resnet152", "vgg", "efficientnet", "convnext", "regnet"]:
    with open('predictions/predictions_{}_3.csv'.format(model_name), 'r') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            slide_id = row['slide_id']
            if slide_id in slide_to_race.keys():
                race = slide_to_race[slide_id]
                races.add(race)
                if race not in predictions:
                    predictions[race] = []
                    actuals[race] = []
                predictions[race].append(int(row['pred_stage']))
                actuals[race].append(int(row['actual_stage']))


    print(model_name)
    for race in races:
        metrics = bootstrap_metrics(actuals[race], predictions[race])
        print(f"Model Name: {model_name}")
        print(f"Race: {race}")
        print(f"Bootstrap Accuracy: {metrics['accuracy']['mean']:.2f}% ± {metrics['accuracy']['std']:.2f}")
        print(f"Bootstrap Precision: {metrics['precision']['mean']:.2f} ± {metrics['precision']['std']:.2f}")
        print(f"Bootstrap Recall: {metrics['recall']['mean']:.2f} ± {metrics['recall']['std']:.2f}")
        print(f"Bootstrap Weighted F1 Score: {metrics['f1']['mean']:.2f} ± {metrics['f1']['std']:.2f}")
        print("-----------------------------")



resnet18
Model Name: resnet18
Race: 1
Bootstrap Accuracy: 37.50% ± 2.24
Bootstrap Precision: 0.36 ± 0.03
Bootstrap Recall: 0.38 ± 0.02
Bootstrap Weighted F1 Score: 0.34 ± 0.03
-----------------------------
Model Name: resnet18
Race: grouped_race
Bootstrap Accuracy: 21.90% ± 1.78
Bootstrap Precision: 0.73 ± 0.03
Bootstrap Recall: 0.22 ± 0.02
Bootstrap Weighted F1 Score: 0.24 ± 0.02
-----------------------------
resnet50
Model Name: resnet50
Race: 1
Bootstrap Accuracy: 34.35% ± 1.83
Bootstrap Precision: 0.40 ± 0.02
Bootstrap Recall: 0.34 ± 0.02
Bootstrap Weighted F1 Score: 0.36 ± 0.02
-----------------------------
Model Name: resnet50
Race: grouped_race
Bootstrap Accuracy: 36.64% ± 1.14
Bootstrap Precision: 0.71 ± 0.02
Bootstrap Recall: 0.37 ± 0.01
Bootstrap Weighted F1 Score: 0.45 ± 0.01
-----------------------------
wide_resnet101
Model Name: wide_resnet101
Race: 1
Bootstrap Accuracy: 32.87% ± 1.26
Bootstrap Precision: 0.39 ± 0.01
Bootstrap Recall: 0.33 ± 0.01
Bootstrap Weighted F1 Sco

In [68]:
# ["resnet18", "resnet50", "wide_resnet101", "resnet152", "vgg", "efficientnet", "convnext", "regnet"]

resnet18 = pd.read_csv("predictions/predictions_resnet18_3.csv")
resnet50 = pd.read_csv("predictions/predictions_resnet50_3.csv")
resnet152 = pd.read_csv("predictions/predictions_resnet152_3.csv")
efficientnet = pd.read_csv("predictions/predictions_efficientnet_3.csv")
convnext = pd.read_csv("predictions/predictions_convnext_3.csv")
wide_resnet = pd.read_csv("predictions/predictions_wide_resnet101_3.csv")
vgg = pd.read_csv("predictions/predictions_vgg_3.csv")
regnet = pd.read_csv("predictions/predictions_regnet_3.csv")

slide_ids = resnet50.slide_id.tolist()
actual_stages = resnet50.actual_stage.tolist()
biopsy_ids = resnet50.biopsy_id.tolist()
races = resnet50.race.tolist()
rest18_preds = resnet18.pred_stage.tolist()
rest50_preds = resnet50.pred_stage.tolist()
rest152_preds = resnet152.pred_stage.tolist()
efficientnet_preds = efficientnet.pred_stage.tolist()
convnext_preds = convnext.pred_stage.tolist()
wide_resnet_preds = wide_resnet.pred_stage.tolist()
vgg_preds = vgg.pred_stage.tolist()
regnet_preds = regnet.pred_stage.tolist()


A = [rest50_preds, efficientnet_preds, convnext_preds, wide_resnet_preds, vgg_preds, regnet_preds, rest18_preds, rest152_preds]

# Using only models that have loss < 1
avg_preds = [round((r50 + effnet + convnext_pred + wide_resnet_pred + vgg_pred + regnet_pred + rest18_pred + rest152_pred)/8)
             for r50, effnet, convnext_pred, wide_resnet_pred, vgg_pred, regnet_pred, rest18_pred, rest152_pred in 
             zip(rest50_preds, efficientnet_preds, convnext_preds, wide_resnet_preds, vgg_preds, regnet_preds, rest18_preds, rest152_preds)]

final_frame = pd.DataFrame()
final_frame['slide_id'] = slide_ids
final_frame["biopsy_id"] = biopsy_ids
final_frame['pred_stage'] = avg_preds
final_frame["actual_stage"] = actual_stages
final_frame["race"] = races
print(final_frame.actual_stage.unique())
aggregated_df = final_frame.groupby('biopsy_id').agg({
    'pred_stage': 'mean',
    'slide_id': 'first',
    'actual_stage': 'first',
    'race': 'first'
}).reset_index()

# Round the mean value to the nearest integer
aggregated_df['pred_stage'] = aggregated_df['pred_stage'].round().astype(int)

# print(aggregated_df.head())
aggregated_df.to_csv('predictions/predictions_final_3.csv', index=False)
print(aggregated_df.actual_stage.unique())

[0 1 2]
[2 0 1]


In [25]:
from PIL import Image
from tqdm import tqdm
import csv
import pandas as pd
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Function to group races
def group_race(race):
    if race in ['2', '3', '4', '8', '9']:
        return 'grouped_race'
    else:
        return race

slide_to_race = {}
with open('metadata/data.csv', 'r') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        slide_to_race[row['slide_id']] = group_race(row['race'])

predictions = {}
actuals = {}
races = set()
with open('predictions/predictions_final_3.csv') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            slide_id = row['slide_id']
            if slide_id in slide_to_race.keys():
                race = slide_to_race[slide_id]
                races.add(race)
                if race not in predictions:
                    predictions[race] = []
                    actuals[race] = []
                predictions[race].append(int(row['pred_stage']))
                actuals[race].append(int(row['actual_stage']))

for race in races:
    metrics = bootstrap_metrics(actuals[race], predictions[race])
    print(f"Model Name: {model_name}")
    print(f"Race: {race}")
    print(f"Bootstrap Accuracy: {metrics['accuracy']['mean']:.2f}% ± {metrics['accuracy']['std']:.2f}")
    print(f"Bootstrap Precision: {metrics['precision']['mean']:.2f} ± {metrics['precision']['std']:.2f}")
    print(f"Bootstrap Recall: {metrics['recall']['mean']:.2f} ± {metrics['recall']['std']:.2f}")
    print(f"Bootstrap Weighted F1 Score: {metrics['f1']['mean']:.2f} ± {metrics['f1']['std']:.2f}")
    print("-----------------------------")

/opt/venv/default/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/venv/default/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/venv/default/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/venv/default/lib/python3.10/site-packages/sklearn/metrics/_classification.py:

Model Name: regnet
Race: 1
Bootstrap Accuracy: 56.72% ± 6.26
Bootstrap Precision: 0.44 ± 0.08
Bootstrap Recall: 0.57 ± 0.06
Bootstrap Weighted F1 Score: 0.49 ± 0.07
-----------------------------
Model Name: regnet
Race: grouped_race
Bootstrap Accuracy: 69.23% ± 6.41
Bootstrap Precision: 0.53 ± 0.08
Bootstrap Recall: 0.69 ± 0.06
Bootstrap Weighted F1 Score: 0.60 ± 0.08
-----------------------------


/opt/venv/default/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/venv/default/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/venv/default/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
